In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import os

print("✅ Biblioteki załadowane")

✅ Biblioteki załadowane


In [2]:
def convert_time_to_seconds(time):
    """Konwertuje czas HH:MM:SS na sekundy"""
    if pd.isnull(time) or time in ['DNS', 'DNF']:
        return None
    parts = str(time).split(':')
    if len(parts) == 3:
        return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
    return None

def extract_age_from_category(category):
    """Wyciąga wiek z kategorii wiekowej (np. 'M30' -> 30)"""
    if pd.isnull(category):
        return None
    age_str = ''.join(filter(str.isdigit, str(category)))
    if age_str:
        return int(age_str)
    return None

print("✅ Funkcje pomocnicze zdefiniowane")

✅ Funkcje pomocnicze zdefiniowane


In [4]:
# Wczytaj dane (ścieżka względna z folderu notebooks/)
df_2023 = pd.read_csv('../data/halfmarathon_wroclaw_2023__final.csv', sep=';')
df_2024 = pd.read_csv('../data/halfmarathon_wroclaw_2024__final.csv', sep=';')

# Połącz dane z obu lat
df = pd.concat([df_2023, df_2024], ignore_index=True)

print(f"✅ Dane wczytane:")
print(f"   - 2023: {len(df_2023)} rekordów")
print(f"   - 2024: {len(df_2024)} rekordów")
print(f"   - Łącznie: {len(df)} rekordów")

✅ Dane wczytane:
   - 2023: 8950 rekordów
   - 2024: 13007 rekordów
   - Łącznie: 21957 rekordów


In [5]:
# Konwertuj czas na sekundy
df['Czas_sekundy'] = df['Czas'].apply(convert_time_to_seconds)

# Wyciągnij wiek z kategorii wiekowej
df['Wiek'] = df['Kategoria wiekowa'].apply(extract_age_from_category)

# Płeć: M -> 1, K -> 0
df['Płeć_encoded'] = df['Płeć'].map({'M': 1, 'K': 0})

# Tempo 5km
df['5km_tempo'] = df['5 km Tempo']

# Usuń rekordy z brakującymi danymi
df_clean = df.dropna(subset=['Czas_sekundy', 'Wiek', 'Płeć_encoded', '5km_tempo'])

print(f"✅ Przed czyszczeniem: {len(df)} rekordów")
print(f"✅ Po czyszczeniu: {len(df_clean)} rekordów")
print(f"❌ Usunięto: {len(df) - len(df_clean)} rekordów z brakami")

print(f"\n📊 Statystyki:")
print(f"   Wiek: {df_clean['Wiek'].min():.0f} - {df_clean['Wiek'].max():.0f}")
print(f"   Czas: {df_clean['Czas_sekundy'].min():.0f}s - {df_clean['Czas_sekundy'].max():.0f}s")
print(f"   Tempo 5km: {df_clean['5km_tempo'].min():.2f} - {df_clean['5km_tempo'].max():.2f} min/km")

✅ Przed czyszczeniem: 21957 rekordów
✅ Po czyszczeniu: 18393 rekordów
❌ Usunięto: 3564 rekordów z brakami

📊 Statystyki:
   Wiek: 20 - 80
   Czas: 3843s - 12754s
   Tempo 5km: 0.00 - 12.75 min/km


In [6]:
# Features: płeć, wiek, tempo_5km
X = df_clean[['Płeć_encoded', 'Wiek', '5km_tempo']].values
y = df_clean['Czas_sekundy'].values

# Podział train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✅ Features przygotowane:")
print(f"   Shape X: {X.shape}")
print(f"   Shape y: {y.shape}")
print(f"\n✅ Podział danych:")
print(f"   Train: {len(X_train)} rekordów")
print(f"   Test:  {len(X_test)} rekordów")
print(f"\n📊 Przykład:")
print(f"   Features [Płeć, Wiek, Tempo_5km]: {X[0]}")
print(f"   Target (Czas): {y[0]:.0f}s ({y[0]/60:.1f} min)")

✅ Features przygotowane:
   Shape X: (18393, 3)
   Shape y: (18393,)

✅ Podział danych:
   Train: 14714 rekordów
   Test:  3679 rekordów

📊 Przykład:
   Features [Płeć, Wiek, Tempo_5km]: [ 1.         30.          2.92333333]
   Target (Czas): 3899s (65.0 min)


In [7]:
# Model: Linear Regression
print("🤖 Trenowanie Linear Regression...")

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predykcja
lr_pred = lr_model.predict(X_test)

# Metryki
lr_mae = mean_absolute_error(y_test, lr_pred)
lr_r2 = r2_score(y_test, lr_pred)

print(f"\n📊 Wyniki Linear Regression:")
print(f"   MAE: {lr_mae:.2f} sekund = {lr_mae/60:.2f} minut")
print(f"   R²:  {lr_r2:.4f}")

print(f"\n🧪 Przykładowe predykcje (pierwsze 5):")
for i in range(5):
    real = y_test[i] / 60
    pred = lr_pred[i] / 60
    diff = abs(real - pred)
    print(f"   {real:.1f} min → {pred:.1f} min (różnica: {diff:.1f} min)")

🤖 Trenowanie Linear Regression...

📊 Wyniki Linear Regression:
   MAE: 298.74 sekund = 4.98 minut
   R²:  0.8889

🧪 Przykładowe predykcje (pierwsze 5):
   161.2 min → 149.8 min (różnica: 11.4 min)
   115.7 min → 112.9 min (różnica: 2.7 min)
   111.9 min → 115.1 min (różnica: 3.2 min)
   123.8 min → 125.3 min (różnica: 1.5 min)
   115.2 min → 110.7 min (różnica: 4.6 min)


In [8]:
# Model: Random Forest
print("🌲 Trenowanie Random Forest...")

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predykcja
rf_pred = rf_model.predict(X_test)

# Metryki
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print(f"\n📊 Wyniki Random Forest:")
print(f"   MAE: {rf_mae:.2f} sekund = {rf_mae/60:.2f} minut")
print(f"   R²:  {rf_r2:.4f}")

# Porównanie
print(f"\n🏆 Porównanie modeli:")
print(f"   Linear Regression: MAE = {lr_mae/60:.2f} min, R² = {lr_r2:.4f}")
print(f"   Random Forest:     MAE = {rf_mae/60:.2f} min, R² = {rf_r2:.4f}")

if lr_mae < rf_mae:
    best_model = lr_model
    best_name = "Linear Regression"
    best_mae = lr_mae
    print(f"\n✅ ZWYCIĘZCA: Linear Regression")
else:
    best_model = rf_model
    best_name = "Random Forest"
    best_mae = rf_mae
    print(f"\n✅ ZWYCIĘZCA: Random Forest")

🌲 Trenowanie Random Forest...

📊 Wyniki Random Forest:
   MAE: 302.94 sekund = 5.05 minut
   R²:  0.8832

🏆 Porównanie modeli:
   Linear Regression: MAE = 4.98 min, R² = 0.8889
   Random Forest:     MAE = 5.05 min, R² = 0.8832

✅ ZWYCIĘZCA: Linear Regression


In [9]:
# Zapisz najlepszy model do folderu models/
import os

# Upewnij się że folder models/ istnieje
os.makedirs('../models', exist_ok=True)

model_path = '../models/model.pkl'
joblib.dump(best_model, model_path)

print(f"✅ Model zapisany:")
print(f"   Lokalizacja: {os.path.abspath(model_path)}")
print(f"   Typ: {best_name}")
print(f"   MAE: {best_mae/60:.2f} minut")
print(f"   R²: {lr_r2:.4f}")
print(f"   Features: [Płeć_encoded, Wiek, Tempo_5km]")

# Sprawdź rozmiar
file_size = os.path.getsize(model_path)
print(f"   Rozmiar: {file_size / 1024:.2f} KB")

print(f"\n✅ Model gotowy do użycia w aplikacji!")

✅ Model zapisany:
   Lokalizacja: C:\Users\Dom\Desktop\DATA_od_zera_do_ai\modul_9_halfmarathon-final\models\model.pkl
   Typ: Linear Regression
   MAE: 4.98 minut
   R²: 0.8889
   Features: [Płeć_encoded, Wiek, Tempo_5km]
   Rozmiar: 0.56 KB

✅ Model gotowy do użycia w aplikacji!
